# Inheco Incubator (Shaker)

<table style="width:100%; border-collapse:collapse;">
<tr>
<td style="width:60%; font-size:15px; line-height:1.7; vertical-align:top; padding-right:15px;">

<ul style="margin-top:0;">
  <li><a href="https://www.inheco.com/incubator-shaker.html" target="_blank"><b>OEM Link</b></a></li>
  <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB-A</li>
  <li><b>Communication Level:</b> Firmware (documentation shared by OEM)</li>
  <li>Same command set for:
    <ul>
      <li>Incubator “MP”</li>
      <li>Incubator “DWP”</li>
      <li>Incubator Shaker “MP”</li>
      <li>Incubator Shaker “DWP”</li>
    </ul>
  </li>
  <li><b>Incubator Shaker “MP”</b> VID:PID <code>0403:6001</code></li>
  <li>Takes in a single plate via a loading tray, heats it to the set temperature, and shakes it to the set RPM.</li>
</ul>

</td>

<td style="width:40%; text-align:center; vertical-align:middle;">
  <img src="img/inheco_incubator_shaker_mp_dwp.png" width="500"/><br>
  <i>Figure: Inheco Incubator Shaker MP & DWP models</i>
</td>
</tr>
</table>

## About the Device(s)

Inheco incubator shakers are modular devices used for temperature control and shaking in lab automation setups.
They differentiate themselves:
- **heater shakers** ... heat a material on which a plate is being placed; open-access; non-uniform temperature distribution around the plate; enables shaking of plate.
- **incubator shakers** ... an enclosed chamber that is being heated and houses a plate; plate access is controlled via a loading tray and a door; *highly uniform temperature distribution around the plate*; enables shaking of plate.

The Inheco incubator devices come in 4 versions, dependent on (1) whether they provide a shaking feature & (2) the size of plates they accept:


| **RTS Code** | **Shaking Feature** | **Plate Format** | **Device Identifier** | **Typical Model** |
|:-------------:|:--------------:|:----------------:|:----------------------|:------------------|
| `0` | ❌ No | MP (Microplate) | `incubator_mp` | INHECO Incubator MP | 
| `1` | ✅ Yes | MP (Microplate) | `incubator_shaker_mp` | INHECO Incubator Shaker MP | 
| `2` | ❌ No | DWP (Deepwell Plate) | `incubator_dwp` | INHECO Incubator DWP | 
| `3` | ✅ Yes | DWP (Deepwell Plate) | `incubator_shaker_dwp` | INHECO Incubator Shaker DWP | 


```{note}
Note: All 4 machines can be controlled with the same PyLabRobot Backend, called `InhecoIncubatorShakerBackend`!
```

---
## Setup Instructions (Physical)

<!-- ![quadrants](img/inheco_incubator_shaker_physical_setup_overview.png) -->
<table style="width:100%; border-collapse:collapse; margin-top:10px;">
<tr>
<td style="text-align:center; vertical-align:middle;">
  <img src="img/inheco_incubator_shaker_physical_setup_overview.png" width="950" style="border-radius:8px;"/>
  <br>
  <i>Figure: Physical setup overview of the Inheco Incubator Shaker system</i>
</td>
</tr>
</table>


To facilitate integration, multiple devices can be placed on top of each other to form an Incubator Shaker Stack (see infographic above).
Up to 6 machines can be placed into the same stack. 
When using more than 6 machines, you must build multiple stacks (none can contain more than 6 machines).
The machines in a single stack can be of any of the 4 types.

The benefit of this setup is that only **one** power cable and only **one** USB cable have to be plugged into the machine at the very bottom of a machine (i.e. stack index 0).
Machines above the bottom one only need to be connected with the machine below it using the 15-pin SUB-D connectors that come with each machine when bought from Inheco.

```{note}
Note: In PyLabRobot, each machine is controlled via its own instance of the `InhecoIncubatorShakerBackend`.
```

<table>
<tr>
<td style="font-size:15px; line-height:1.6; width:60%; vertical-align:top; padding-right:10px;">

To connect an <code>InhecoIncubatorShakerBackend</code> there are two identifiers that uniquely characterise every physical machine:<br><br>

<ol style="margin-left: 20px;">
<li><b>DIP switch identifier</b> — located on the back of the bottom machine; it defines the DIP switch configuration for the entire stack above it.<br>
(<i>Note: You must set this DIP switch manually; see instructions below.</i>)</li>
<li><b>Stack index number</b> — the position a machine occupies within its stack.<br>
(<i>Note: This cannot be reassigned after connecting the stack unless the physical stack arrangement is changed.</i>)</li>
</ol>

<h3>Setting the DIP switch to generate a machine address</h3>

The DIP switch at the back of each machine consists of 4 pins that can be set into an <code>UP</code> / <code>0</code> or a <code>DOWN</code> / <code>1</code> position.

(<i>Note:</b> There are two more pins to the left of the DIP switch pins. They are not involved in setting the DIP switch address, and should be left in their <code>DOWN</code> position.</i>)</li>

This represents <i>binary encoding</i>:
<ul>
<li>All pins at <code>0</code> → DIP switch is set to address <code>0</code></li>
<li>All pins at <code>1</code> → DIP switch is set to address <code>15</code> (2<sup>4</sup>-1)</li>
</ul>

</td>

<td style="width:40%; text-align:center; vertical-align:middle;">
<img src="img/inheco_incubator_shaker_dip_switch_addressing.png" width="500"/><br>
<i>Figure: DIP switch layout to generate different identifiers/addresses</i>
</td>
</tr>
</table>


---
## Setup Instructions (Programmatic)

After the two cables have been connected to the bottom Inheco Incubator Shaker, you only have to instantiate the `InhecoIncubatorShakerBackend` and give it the correct `dip_switch_id` & `stack_index`:

In [1]:
import asyncio
import logging
from typing import Optional, Literal, Dict, Any

from pylabrobot.io.serial import Serial

try:
  import serial
  import serial.tools.list_ports
  HAS_SERIAL = True
except ImportError as e:
  HAS_SERIAL = False
  _SERIAL_IMPORT_ERROR = e


class InhecoError(RuntimeError):
    """Represents an INHECO firmware-reported error."""

    def __init__(self, command: str, code: str, message: str):
        super().__init__(f"{command} failed with error {code}: {message}")
        self.command: str = command
        self.code: str = code
        self.message: str = message


_REF_FLAG_NAMES: Dict[int, str]  = {
    # Heater (0–15) — names per manual’s heater flags table (subset shown here)
    0:  "H_WARN_WarmUp_TIME",
    1:  "H_WARN_BoostCoolDown_TIME",
    2:  "H_WARN_StartState_LIMIT_Up_TEMP_S2",
    3:  "H_WARN_StartState_LIMIT_Up_TEMP_S3",
    4:  "H_WARN_StartStateBoost_LIMIT_UpDown_TEMP_S3",
    5:  "H_WARN_StableState_LIMIT_UpDown_TEMP_S2",
    6:  "H_WARN_StableState_LIMIT_UpDown_TEMP_S3",
    7:  "H_WARN_DELTA_TEMP_S1_S2",
    8:  "H_ERR_DELTA_TEMP_S1_S2",
    9:  "H_WARN_StartStateBoost_LIMIT_UpDown_TEMP_S2",
    10: "H_WARN_WaitStable_LIMIT_TEMP_S1",
    11: "H_WARN_WaitStable_LIMIT_TEMP_S2",
    12: "H_WARN_WaitStable_LIMIT_TEMP_S3",
    13: "H_ERR_S2_NTC_NotConnected",
    14: "H_ERR_S3_NTC_NotConnected",
    15: "H_WARN_DELTA_TEMP_S1_S3",

    # Shaker (16–26) — names per manual’s shaker flag set (page 39)
    16: "S_WARN_MotorCurrentLimit",
    17: "S_WARN_TargetSpeedTimeout",
    18: "S_WARN_PositionTimeout",
    19: "S_ERR_MotorTemperatureLimit",
    20: "S_ERR_TargetSpeedDeviation",
    21: "S_ERR_HomeSensorTimeout",
    22: "S_ERR_MotorDriverFault",
    23: "S_ERR_EncoderSignalLost",
    24: "S_ERR_AmplitudeOutOfRange",
    25: "S_ERR_VibrationExcessive",
    26: "S_ERR_InternalTimeout",
    # 27–31 reserved
}

FIRMWARE_ERROR_MAP: Dict[int, str] = {
    0: "Msg Ok",
    1: "Reset detected",
    2: "Invalid command",
    3: "Invalid operand",
    4: "Protocol error",
    5: "Reserved",
    6: "Timeout from Device",
    7: "Device not initialized",
    8: "Command not executable",
    9: "Drawer not in end position",
    10: "Unexpected Labware Status",
    13: "Drawer DWP not perfectly closed (NTC not connected)",
    14: "Floor ID error",
    15: "Timeout sub device",
}


class InhecoIncubatorShakerBackend:
    """
    Asynchronous backend for controlling an INHECO Incubator/Shaker via USB-VCP.

    Handles auto-detection, asynchronous serial I/O, command encoding/decoding,
    and structured firmware error reporting.

    Example:
        ```python
        incubator = InhecoIncubatorShakerBackend(dip_switch_id=2)
        await incubator.setup(verbose=True)
        await incubator.set_temperature(37.0)
        await incubator.stop()
        ```
    """

    # === Logging ===

    def _log(self, level: int, message: str, direction: Optional[str] = None):
        """
        Unified logging with a clear device tag and optional direction marker.
        direction: "→" for TX, "←" for RX, None for neutral.
        """
        prefix = f"[INHECO IncShak dip={self.dip_switch_id} stack={self.stack_index}]"
        if direction:
            prefix += f" {direction}"
        self.logger.log(level, f"{prefix} {message}")

    # === Constructor ===

    def __init__(
        self,
        port: Optional[str] = None,
        dip_switch_id: int = 2,
        stack_index: int = 0,
        write_timeout: float = 5.0,
        read_timeout: float = 10.0,
        logger: Optional[logging.Logger] = None,
    ) -> None:
        """Prepare backend instance. Serial link is opened asynchronously in `setup()`."""
    
        # Logger
        self.logger = logger or logging.getLogger("pylabrobot")
        self.logger.setLevel(logging.INFO)
        logging.getLogger("pylabrobot.io.serial").disabled = True
    
        # Core state
        self.dip_switch_id = dip_switch_id
        self.stack_index = stack_index
        # self.ser: Optional[Serial] = None
        self.write_timeout = write_timeout
        self.read_timeout = read_timeout
    
        # Defer port resolution to setup()
        self.port_hint = port
    
        # Cached state
        self.setup_finished = False
        self.is_initialized = False
        self.loading_tray = "unknown"
        self.incubator_type = "unknown"
        self.firmware_version = "unknown"

    # === Device probing ===

    async def _probe_inheco_port(self, dev: str, stack_index: int) -> bool:
        """Attempt RTS handshake using pylabrobot.io.serial.Serial (async-safe)."""
        ser = Serial(
            port=dev,
            baudrate=19200,
            timeout=1,
            write_timeout=1,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
        )
        try:
            await ser.setup()
            msg = self._build_message("RTS", stack_index=stack_index)
            await ser.write(msg)
            data = await ser.read(64)
            expected_hdr = (0xB0 + self.dip_switch_id) & 0xFF
            ok = bool(data and data[0] == expected_hdr)
            return ok
        except Exception as e:
            self._log(logging.DEBUG, f"Probe failed on {dev}: {e}")
            return False
        finally:
            try:
                await ser.stop()
            except Exception:
                pass


    # === Lifecycle ===

    async def setup(self, verbose: bool = False) -> None:
        """
        Detect and connect to the INHECO device.

        Probes available FTDI serial devices (VID:PID 0403:6001), validates DIP ID,
        and initializes communication.
        """
        VID = "0403"; PID = "6001"
    
        # --- Explicit port path ---
        # If user gave a port, use it but verify DIP
        if self.port_hint is not None:
            candidate = self.port_hint
            self._log(logging.INFO, f"Using explicitly provided port: {candidate} (verifying DIP={self.dip_switch_id})")
            ok = await self._probe_inheco_port(candidate, self.stack_index)
            if not ok:
                msg = (f"Device on {candidate} did not respond with expected DIP switch "
                       f"ID={self.dip_switch_id}. Please verify the DIP switch setting.")
                self._log(logging.ERROR, msg)
                raise RuntimeError(msg)
            self.port = candidate

        # --- Auto-detect FTDI devices ---
        else:
            matching_ports = [
                p.device
                for p in serial.tools.list_ports.comports()
                if f"{VID}:{PID}" in (p.hwid or "")
            ]
            
            if not matching_ports:
                msg = f"No INHECO FTDI devices found (VID={VID}, PID={PID})."
                self._log(logging.ERROR, msg)
                raise RuntimeError(msg)
    
            if len(matching_ports) == 1:
                candidate = matching_ports[0]
                self._log(logging.INFO, f"Verifying single detected INHECO on {candidate} (DIP={self.dip_switch_id})...")
                ok = await self._probe_inheco_port(candidate, self.stack_index)
                if not ok:
                    msg = (f"Device on {candidate} did not respond with expected DIP switch "
                           f"ID={self.dip_switch_id}. Please verify the DIP switch setting.")
                    self._log(logging.ERROR, msg)
                    raise RuntimeError(msg)
                self.port = candidate
                self._log(logging.INFO, f"Auto-selected {self.port} (DIP {self.dip_switch_id}).")
            
            else:
                self._log(logging.INFO,
                    f"Multiple INHECO FTDI devices found ({len(matching_ports)}). "
                    f"Probing for DIP={self.dip_switch_id}..."
                )
                responsive_ports = []
                for dev in matching_ports:
                    if await self._probe_inheco_port(dev, self.stack_index):
                        responsive_ports.append(dev)
    
                if not responsive_ports:
                    msg = (f"No INHECO responded for dip_switch_id={self.dip_switch_id}, "
                           f"stack_index={self.stack_index}. Verify DIP and connections.")
                    self._log(logging.ERROR, msg)
                    raise RuntimeError(msg)
    
                if len(responsive_ports) > 1:
                    msg = (f"Multiple INHECO devices respond for dip_switch_id={self.dip_switch_id}: "
                           f"{', '.join(responsive_ports)}")
                    self._log(logging.ERROR, msg)
                    raise RuntimeError(msg)
    
                self.port = responsive_ports[0]
                self._log(logging.INFO, f"Auto-selected port {self.port} for DIP {self.dip_switch_id}.")

        # --- Create persistent async serial link with a verified port ---
        self.io = Serial(
            port=self.port,
            baudrate=19200,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=0,  # non-blocking
            write_timeout=self.write_timeout,
        )
        await self.io.setup()
    
        # --- Identify firmware and type ---
        self.firmware_version = await self.request_firmware_version()
        incubator_type = await self.request_incubator_type()
        serial_number = await self.request_serial_number()
    
        msg = (
            f"Connected to INHECO {incubator_type} on {self.port}\n"
            f"Machine serial number: {serial_number}\n"
            f"Firmware version: {self.firmware_version}"
        )
        if verbose:
            print(msg)
        self._log(logging.INFO, msg)
    
        await self.initialize()
        self.setup_finished = True
        self.is_initialized = True


    async def stop(self) -> None:
        """Close the connection and free the serial port."""
        await self.io.stop()
        self._log(logging.INFO, "Disconnected from INHECO Incubator/Shaker")


    # === Low-level I/O ===

    async def write(self, data: bytes) -> None:
        """Write binary data to the serial device."""
        self._log(logging.DEBUG, f"→ {data.hex(' ')}")
        await self.io.write(data)


    async def _read_full_response(self, timeout: float) -> bytes:
        """Read a complete INHECO response frame asynchronously."""
        if not self.io:
            raise RuntimeError("Serial port not open.")
    
        loop = asyncio.get_event_loop()
        start = loop.time()
        buf = bytearray()
        expected_hdr = (0xB0 + self.dip_switch_id) & 0xFF
    
        def has_complete_tail(b: bytearray) -> bool:
            return (
                len(b) >= 3
                and b[-1] == 0x60
                and b[-3] == expected_hdr
                and 0x20 <= b[-2] <= 0x2F
            )
    
        while True:
            # Try to read up to 16 bytes at once — this limits per-byte log spam
            chunk = await self.io.read(16)
            if chunk:
                buf.extend(chunk)
                self._log(logging.DEBUG, chunk.hex(" "), direction="←")
    
                if has_complete_tail(buf):
                    return bytes(buf)
    
            # Timeout protection
            if loop.time() - start > timeout:
                raise TimeoutError(
                    f"Timed out waiting for complete response (so far: {buf.hex(' ')})"
                )
    
            # brief pause to yield to event loop, avoid tight spin
            await asyncio.sleep(0.005)

    
    # === Encoding / Decoding ===

    def _crc8_legacy(self, data: bytearray) -> int:
        """Compute legacy CRC-8 used by INHECO devices.""" # TODO: check remaining combos: shaker[y/n] * size[mp/dwp]
        crc = 0xA1
        for byte in data:
            d = byte
            for _ in range(8):
                if (d ^ crc) & 1:
                    crc ^= 0x18
                    crc >>= 1
                    crc |= 0x80
                else:
                    crc >>= 1
                d >>= 1
        return crc & 0xFF

    def _build_message(self, command: str, stack_index: int = 0) -> bytes:
        """Construct a full binary message with header and CRC."""
        if not (0 <= stack_index <= 5):
            raise ValueError("stack_index must be between 0 and 5")
        cmd = f"T0{stack_index}{command}".encode("ascii")
        length = len(cmd) + 3
        address = (0x30 + self.dip_switch_id) & 0xFF
        proto = (0xC0 + len(cmd)) & 0xFF
        message = bytearray([length, address, proto]) + cmd
        crc = self._crc8_legacy(message)
        return bytes(message + bytearray([crc]))

    def _is_report_command(self, command: str) -> bool:
        """Return True if command is a 'Report' type (starts with 'R')."""

        return command and command[0].upper() == "R"

    # === Response parsing ===

    def _parse_response_binary_safe(self, resp: bytes) -> dict:
        """
        Parse INHECO response frames safely (binary & multi-segment).

        Handles:
          - Set/Action:  [B0+ID][20+err][60]
          - Report:      [B0+ID]<data>[B0+ID]... [B0+ID][20+err][60]
          - Also works when only a single [B0+ID] header precedes data.

        Returns:
          dict(
            device=int,
            error_code=int|None,
            ok=bool,
            data=str,
            raw_data=bytes
          )
        """
        if len(resp) < 3:
            raise ValueError("Incomplete response")

        expected_hdr = (0xB0 + self.dip_switch_id) & 0xFF

        # --- Trim leading junk before the first valid header ---
        try:
            start_idx = resp.index(bytes([expected_hdr]))
            frame = resp[start_idx:]
        except ValueError:
            return {
                "device": None,
                "error_code": None,
                "ok": False,
                "data": "",
                "raw_data": resp,
            }

        # --- Validate tail (status section) ---
        if len(frame) < 3 or frame[-1] != 0x60:
            # No valid tail; may be bootloader or incomplete
            return {
                "device": expected_hdr - 0xB0,
                "error_code": None,
                "ok": False,
                "data": "",
                "raw_data": frame,
            }

        # Extract error code (0x20 + err)
        err_byte = frame[-2]
        err_code = err_byte - 0x20 if 0x20 <= err_byte <= 0x2F else None

        # --- Collect data between headers ---
        # Pattern: [hdr] <data> [hdr] <data> ... [hdr] [20+err][60]
        data_blocks = []
        i = 1  # start right after the first header

        while i < len(frame) - 3:
            try:
                next_hdr = frame.index(bytes([expected_hdr]), i)
            except ValueError:
                # No further header — consume until the status tail
                next_hdr = len(frame) - 3

            # Capture bytes between i and next_hdr
            if next_hdr > i:
                data_blocks.append(frame[i:next_hdr])

            i = next_hdr + 1
            if next_hdr >= len(frame) - 3:
                break

        # --- Assemble and decode ---
        raw_data = b"".join(data_blocks)

        try:
            ascii_data = raw_data.decode("ascii").strip("\x00")
        except UnicodeDecodeError:
            ascii_data = raw_data.hex()

        return {
            "device": expected_hdr - 0xB0,
            "error_code": err_code,
            "ok": (err_code == 0),
            "data": ascii_data,
            "raw_data": raw_data,
        }

    def _is_error_tail(self, resp: bytes) -> bool:
        expected_hdr = (0xB0 + self.dip_switch_id) & 0xFF
        return len(resp) >= 3 and resp.endswith(bytes([expected_hdr, 0x28, 0x60]))


    # === Error Handling ===

    async def debug_error_registry(self):
        """
        Spec-compliant error snapshot using REE/REF/REP.
        - REE: init & labware status (0..3)
        - REF: 32-bit flag mask (bit set = active)
        - REP: heater flag parameters for bits {0..8, 13, 14, 15}
        """
        print("=== ERROR REGISTRY DEBUG ===")
    
        # Use your existing helpers that interpret REE
        try:
            is_init = await self.request_is_initialized()       # uses REE
            plate_known = await self.request_plate_status_known()  # uses REE
            print(f"REE → initialized={is_init}, plate_status_known={plate_known}")
        except Exception as e:
            print(f"REE query failed: {e}")
    
        try:
            ref_raw = await self.send_command("REF")  # returns 32-bit mask as decimal ASCII
            ref_mask = int(ref_raw.strip())
            print(f"REF (flags bitmask): {ref_mask} (0x{ref_mask:08X})")  # 32-bit mask. 
    
            set_bits = [b for b in range(32) if (ref_mask >> b) & 1]
            if not set_bits:
                print(" - No flags set.")
            else:
                print(" - Active flags:")
                for b in set_bits:
                    name = _REF_FLAG_NAMES.get(b, f"Flag{b}")
                    print(f"   [{b:02d}] {name}")
    
                    # REP supports heater selectors {0..8,13,14,15}. 
                    if b in {0,1,2,3,4,5,6,7,8,13,14,15}:
                        try:
                            param = await self.send_command(f"REP{b}")
                            print(f"      → Parameter: {param}")
                        except Exception as e:
                            print(f"      → REP{b} failed: {e}")
    
        except Exception as e:
            print(f"REF/REP read failed: {e}")
    
        print("=== END ERROR REGISTRY DEBUG ===")

    async def _collect_error_context(self) -> dict:
        ctx = {"ree": None, "ref_mask": None, "flags": [], "rep_params": {}}
        try:
            is_init = await self.request_is_initialized()
            plate_known = await self.request_plate_status_known()
            ctx["ree"] = {"initialized": is_init, "plate_status_known": plate_known}
        except Exception:
            pass
    
        try:
            ref_raw = await self.send_command("REF")
            ref_mask = int(ref_raw.strip())
            ctx["ref_mask"] = ref_mask
            set_bits = [b for b in range(32) if (ref_mask >> b) & 1]
            for b in set_bits:
                ctx["flags"].append({"bit": b, "name": _REF_FLAG_NAMES.get(b, f"Flag{b}")})
                if b in {0,1,2,3,4,5,6,7,8,13,14,15}:
                    try:
                        param = await self.send_command(f"REP{b}")
                        ctx["rep_params"][b] = param
                    except Exception:
                        pass
        except Exception:
            pass
    
        return ctx


    # === Command layer ===

    async def send_command(
        self,
        command: str,
        delay: float = 0.2,
        read_timeout: Optional[float] = None,
    ) -> str:
        """
        Send a command to the INHECO device and return its response.

        This method handles binary-safe I/O, firmware error mapping, and structured parsing.
        - Report commands (starting with 'R') return their ASCII/hex payload.
        - Action commands (starting with 'A') return an empty string by default,
          except for 'AQS' (self-test), which returns its raw binary payload bits.

        Args:
            command: Firmware command string, e.g. "RAT1", "STT370", or "AQS".
            delay: Delay between write and read, default 0.2 s.
            read_timeout: Optional custom read timeout in seconds.

        Returns:
            str | bytes: Parsed data field if available (string for report commands,
            raw bytes for AQS), or "" for simple action acknowledgments.

        Raises:
            TimeoutError: If no response was received in time.
            InhecoError: If firmware reports an error (non-zero error tail).
        """
        # === Construct and send message ===
        msg = self._build_message(command, stack_index=self.stack_index)
        self._log(logging.INFO, f"SENT MESSAGE: {msg}")

        await self.write(msg)
        await asyncio.sleep(delay)

        # === Read response frame ===
        response = await self._read_full_response(timeout=read_timeout or self.read_timeout)
        if not response:
            raise TimeoutError(f"No response from machine for command: {command}")

        # === Handle explicit firmware error tails ===
        if self._is_error_tail(response):
            tail_err = response[-2] - 0x20  # 0..15
            code = f"E{tail_err:02d}"
            message = FIRMWARE_ERROR_MAP.get(tail_err, "Unknown firmware error")

            # Optional: Collect diagnostic context for logs and error propagation
            ctx = {}
            try:
                ctx = await self._collect_error_context()
                self._log(logging.DEBUG, f"Error context: {ctx}")
            except Exception:
                pass

            err = InhecoError(command, code, message)
            err.context = ctx
            raise err

        # === Normal parse ===
        self._log(logging.INFO, f"RAW RESPONSE: {response}")
        parsed = self._parse_response_binary_safe(response)
        self._log(logging.DEBUG, f"PARSED RESPONSE: {parsed}")

        # === Handle normal report commands ===
        if self._is_report_command(command):
            if not parsed["ok"]:
                raise InhecoError(command, "E00", "Report returned non-OK status")
            return parsed["data"]

        # === Special-case: AQS returns binary self-test bits ===
        if command.startswith("AQS"):
            if not parsed["ok"]:
                raise InhecoError(command, "E00", "Self-test returned non-OK status")
            # Return raw data bytes if available, else the parsed ASCII field
            if parsed["raw_data"]:
                return parsed["raw_data"]
            return parsed["data"]

        # === Non-report command: verify success ===
        if not parsed["ok"]:
            code_num = parsed.get("error_code")
            if code_num is not None:
                code = f"E{code_num:02d}"
                message = FIRMWARE_ERROR_MAP.get(code_num, "Unknown firmware error")
            else:
                code = "E00"
                message = "Unknown error (no error code reported)"

            ctx = {}
            try:
                ctx = await self._collect_error_context()
                self._log(logging.DEBUG, f"Error context: {ctx}")
            except Exception:
                pass

            err = InhecoError(command, code, message)
            err.context = ctx
            raise err

        # === Return data (if any) or empty string ===
        return parsed["data"] if parsed["data"] else ""



    # === Public high-level API ===

    async def request_firmware_version(self) -> str:
        """Return the firmware version string."""
        return await self.send_command("RFV0")

    async def request_serial_number(self) -> str:
        """Return the device serial number."""
        return await self.send_command("RFV2")

    async def request_last_calibration_date(self) -> str:
        resp = await self.send_command("RCM")
        return resp[:10]

    async def request_number_of_connected_machines(self) -> int:
        resp = await self.send_command("RDA")
        return int(resp)

    async def request_labware_detection_threshold(self) -> int:
        resp = await self.send_command("RDM")
        return int(resp)

    async def request_incubator_type(self) -> str:
        """Return a descriptive string of the incubator/shaker configuration."""

        incubator_type_dict = {
            "0": "incubator_mp",         # no shaker
            "1": "incubator_shaker_mp",
            "2": "incubator_dwp",        # no shaker
            "3": "incubator_shaker_dwp",
        }
        resp = await self.send_command("RTS")
        ident = incubator_type_dict.get(resp, "unknown")
        self.incubator_type = ident
        return ident

    async def request_plate_in_incubator(self) -> bool:
        resp = await self.send_command("RLW")
        return resp == "1"

    async def request_operation_time_in_hours(self) -> int:
        resp = await self.send_command("RDC1")
        return int(resp)

    async def request_drawer_cycles_performed(self) -> int:
        resp = await self.send_command("RDC2")
        return int(resp)

    async def request_is_initialized(self) -> bool:
        resp = await self.send_command("REE")
        return resp in {"0", "2"}

    async def request_plate_status_known(self) -> bool:
        resp = await self.send_command("REE")
        return resp in {"0", "1"}

    async def initialize(self) -> str:
        """Perform device initialization (AID)."""
        return await self.send_command("AID")

    # Drawer

    async def open(self) -> None:
        """Open the incubator door & move loading tray out."""
        await self.send_command("AOD")
        self.loading_tray = "open"

    async def close(self) -> None:
        """Move the loading tray in & close the incubator door."""
        await self.send_command("ACD")
        self.loading_tray = "closed"

    # Temperature

    async def set_temperature(self, temperature: float, passive: bool = False):
        """Set the target incubation temperature in °C."""
        target = round(temperature * 10)
        await self.send_command(f"STT{target}")
        if not passive:
            await self.send_command("SHE1")

    async def stop_temperature(self):
        await self.send_command("SHE0")

    async def get_temperature(
        self,
        sensor: Literal["mean", "main", "dif", "boost"] = "mean",
        ) -> float:
        """Return current measured temperature in °C."""

        sensor_mapping = {
            "mean": [1, 2, 3],
            "main": [1],
            "dif": [2],
            "boost": [3],
        }
        vals = []
        for idx in sensor_mapping[sensor]:
            val = await self.send_command(f"RAT{idx}", read_timeout=60)
            vals.append(int(val) / 10.0)
        return round(sum(vals) / len(vals), 2)

    async def wait_for_temperature(self, timeout: float = 300.0, tolerance: float = 0.5):
        raise NotImplementedError("not yet implemented")

    # Self-Test

    async def perform_self_test(self, read_timeout: int = 500) -> str:
        """Execute the internal self-test routine."""
        
        plate_in = await self.send_command("RLW")
        if plate_in == "1":
            raise ValueError("Self-test requires an empty incubator.")

        elif self.loading_tray == "open":
            raise ValueError("Self-test requires a closed loading tray.")

        return await self.send_command("AQS", read_timeout=read_timeout)




In [2]:
import logging
from pylabrobot.io import LOG_LEVEL_IO
from datetime import datetime
import numpy as np

current_date = datetime.today().strftime('%Y-%m-%d')
protocol_mode = "execution"

# Create the shared file handler once
fh = logging.FileHandler(f"{current_date}_testing_{protocol_mode}.log", mode="a")
fh.setLevel(LOG_LEVEL_IO)
formatter = logging.Formatter(
    "%(asctime)s [%(levelname)s] %(name)s - %(message)s"
)
fh.setFormatter(formatter)

# Configure the main pylabrobot logger
logger_plr = logging.getLogger("pylabrobot")
logger_plr.setLevel(LOG_LEVEL_IO)
if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
           for h in logger_plr.handlers):
    logger_plr.addHandler(fh)

# Other loggers can reuse the same file handler
logger_manager = logging.getLogger("manager")
logger_device = logging.getLogger("device")

for logger in [logger_manager, logger_device]:
    logger.setLevel(logging.DEBUG)  # or logging.INFO
    if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
               for h in logger.handlers):
        logger.addHandler(fh)

# START LOGGING
logger_manager.info("START AUTOMATED PROTOCOL")


In [3]:
# Autodetect FTDI device
incubator = InhecoIncubatorShakerBackend(
    dip_switch_id=2,
    stack_index=0
)
await incubator.setup(verbose=True)

Connected to INHECO incubator_shaker_mp on /dev/cu.usbserial-130
Machine serial number: 2013
Firmware version: IncShak_C_V3.50_04/2012


In [ ]:
incubator.

In [ ]:
await incubator.get_temperature(sensor="main")

In [5]:
await incubator.set_temperature(37)

In [7]:
import time

await incubator.stop_temperature()

for x in range(20):
    
    resp = await incubator.get_temperature(sensor="main")
    print(resp)

    time.sleep(1)

36.9
36.7
36.4
36.0
35.7
35.4
35.1
34.8
34.6
34.4
34.1
34.0
33.8
33.6
33.4
33.3
33.1
32.9
32.8
32.6


In [4]:
await incubator.perform_self_test()

b'0'

In [10]:
await incubator.request_plate_in_incubator()

False

In [11]:
await incubator.debug_error_registry()

=== ERROR REGISTRY DEBUG ===
REE → initialized=True, plate_status_known=True
REF (flags bitmask): 0 (0x00000000)
 - No flags set.
=== END ERROR REGISTRY DEBUG ===


## Usage

### Testing & Investigation

In [12]:
await incubator.send_command("REE")

'0'

In [13]:
await incubator.send_command("RDC2")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

'1675'

'1'

In [16]:
for x in [
    0, 1, 2, 3, 
    5, 7, 10, 15,
    20, 21, 22, 23,
    24, 25
]:
    print(x, await incubator.send_command(f"RFV{x}") )

# Reports the date and an alphanumeric string (e.g. operator) of the last calibration for the device.
# The Data is reported in the Format YYYY-MM-DD,xxxxx (Example: 2005-09-28,xxxxx).
# The five xs are alphanumeric wildcards.

0 IncShak_C_V3.50_04/2012
1 BOOT_C_V3.12_05/2007
2 2013
3 0
5 COPYRIGHT INHECO
7 1
10 IncShak
15 350
20 325
21 315
22 IncShak_R_V3.25_12/2011
23 BOOT_R_V1.10_01/2007
24 IncShak_S_V3.15_05/2011
25 BOOT_S_V1.10_01/2007


'incubator_shaker_mp'

In [9]:
await incubator.stop()

In [15]:

def request_machine_initialisation_status():
    """ """
    resp = await incubator.send_command("REE")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

'0'

### Loading Drawer

In [14]:
await incubator.open_lid()
# await asyncio.sleep(5)
# await incubator.close_lid()

In [16]:
await incubator.close_lid()

### Temperature Control

In [5]:
import time

for t in range(5):
    temp_meas = {}
    
    for x in range(1,4):
        temp_meas[x] = await incubator.send_command(f"RAT{x}", read_timeout=60)

    print(t, temp_meas)
    time.sleep(1)

# This command reports the actual ambient or slot temperatures of the three sensors.
# It can be chosen whether the evaluated temperature or the sensor value should be reported.
# The temperatures are reported in 1/10 °C: 345 = 34,5 °C

0 {1: '184', 2: '186', 3: '187'}
1 {1: '184', 2: '186', 3: '186'}
2 {1: '184', 2: '186', 3: '186'}
3 {1: '184', 2: '186', 3: '186'}
4 {1: '184', 2: '186', 3: '186'}


In [ ]:
    async def get_temperature(self, sensor: Literal["all", "main", "dif", "boos"]) -> float:
        """Get the current temperature of the temperature controller in Celsius."""

        sensor_mapping = {
            "all": [1, 2, 3],
            "main": [1],
            "dif": [2],
            "boos": [3]
        }

        resp_summary = []
        
        for idx in sensor_mapping[sensor]:
            
            temp_meas = await incubator.send_command(f"RAT{x}", read_timeout=60)

            resp_summary.append(int(temp_meas)/10)      
        
        raise resp_summary/len(resp_summary)

In [ ]:
await incubator.send_command("RTT", read_timeout=60)


In [ ]:
await incubator.send_command("STT700", read_timeout=60)


### Shaking Control

TODO

In [4]:
import serial.tools.list_ports

def list_serial_devices():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        print(f"Device: {port.device}")
        print(f"  Description: {port.description}")
        print(f"  HWID: {port.hwid}")
        print("-" * 40)

if __name__ == "__main__":
    print("Scanning USB serial devices...")
    list_serial_devices()


Scanning USB serial devices...
Device: /dev/cu.debug-console
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.BoseQC35II
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.Bluetooth-Incoming-Port
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.usbserial-140
  Description: USB <-> Serial
  HWID: USB VID:PID=0403:6001 LOCATION=0-1.4
----------------------------------------


### Closing Connection

In [5]:
await incubator.stop()